# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [103]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [104]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,tarakan,3.3000,117.6333,30.72,79,40,2.57,ID,1700375448
1,1,fort bragg,35.1390,-79.0060,12.68,57,0,3.60,US,1700375553
2,2,ancud,-41.8697,-73.8203,9.84,88,100,2.79,CL,1700375553
3,3,adamstown,-25.0660,-130.1015,21.82,86,100,9.88,PN,1700375553
4,4,hovd,48.0056,91.6419,2.55,68,90,1.25,MN,1700375554


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [105]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = True,
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5
)

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [106]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_narrow_df = city_data_df[(city_data_df['Max Temp'] < 27) &
                             (city_data_df['Max Temp'] > 21) &
                             (city_data_df['Wind Speed'] < 4.5) &
                             (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
city_data_narrow_df = city_data_narrow_df.dropna()

# Display sample data
city_data_narrow_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
23,23,praia,14.9215,-23.5087,24.30,60,0,4.12,CV,1700375227
43,43,haiku-pauwela,20.9219,-156.3051,23.36,79,0,1.54,US,1700375567
50,50,saint-philippe,-21.3585,55.7679,26.77,63,0,4.13,RE,1700375569
76,76,katsuren-haebaru,26.3369,127.8719,22.21,46,0,4.12,JP,1700375578
83,83,cabo san lucas,22.8909,-109.9124,26.95,83,0,0.45,MX,1700375580
137,137,mazatlan,23.2167,-106.4167,26.62,82,0,0.45,MX,1700375599
161,161,karachi,24.9056,67.0822,25.90,31,0,3.60,PK,1700375569
209,209,tazacorte,28.6290,-17.9293,22.30,32,0,4.47,ES,1700375622
254,254,san patricio,28.0170,-97.5169,21.92,84,0,1.43,US,1700375640
255,255,rampur,28.8167,79.0333,26.28,36,0,3.46,IN,1700375594


### Step 3: Create a new DataFrame called `hotel_df`.

In [107]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_narrow_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
23,praia,CV,14.9215,-23.5087,60,
43,haiku-pauwela,US,20.9219,-156.3051,79,
50,saint-philippe,RE,-21.3585,55.7679,63,
76,katsuren-haebaru,JP,26.3369,127.8719,46,
83,cabo san lucas,MX,22.8909,-109.9124,83,
137,mazatlan,MX,23.2167,-106.4167,82,
161,karachi,PK,24.9056,67.0822,31,
209,tazacorte,ES,28.6290,-17.9293,32,
254,san patricio,US,28.0170,-97.5169,84,
255,rampur,IN,28.8167,79.0333,36,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [108]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
     "apiKey":geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
praia - nearest hotel: Casa Sodadi
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
saint-philippe - nearest hotel: Le Baril
katsuren-haebaru - nearest hotel: No hotel found
cabo san lucas - nearest hotel: Comfort Rooms
mazatlan - nearest hotel: Mona Inn
karachi - nearest hotel: Quetta Agha Hotel
tazacorte - nearest hotel: App Leyma
san patricio - nearest hotel: No hotel found
rampur - nearest hotel: تمنا ہوٹل اور ریستوران
venice - nearest hotel: El Patio
mirganj - nearest hotel: No hotel found
taco pozo - nearest hotel: Hospedaje El Camionero
dalbandin - nearest hotel: SF Al-dawood Restaurant


,City,Country,Lat,Lng,Humidity,Hotel Name
23,praia,CV,14.9215,-23.5087,60,Casa Sodadi
43,haiku-pauwela,US,20.9219,-156.3051,79,Inn At Mama's Fish House
50,saint-philippe,RE,-21.3585,55.7679,63,Le Baril
76,katsuren-haebaru,JP,26.3369,127.8719,46,No hotel found
83,cabo san lucas,MX,22.8909,-109.9124,83,Comfort Rooms
137,mazatlan,MX,23.2167,-106.4167,82,Mona Inn
161,karachi,PK,24.9056,67.0822,31,Quetta Agha Hotel
209,tazacorte,ES,28.6290,-17.9293,32,App Leyma
254,san patricio,US,28.0170,-97.5169,84,No hotel found
255,rampur,IN,28.8167,79.0333,36,تمنا ہوٹل اور ریستوران


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [109]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = True,
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    hover_cols = ["Hotel Name","Country"]
)

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)